In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import joblib


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/traindemographics.csv')
print(data.head())

                         customerid                   birthdate  \
0  8a858e135cb22031015cbafc76964ebd  1973-10-10 00:00:00.000000   
1  8a858e275c7ea5ec015c82482d7c3996  1986-01-21 00:00:00.000000   
2  8a858e5b5bd99460015bdc95cd485634  1987-04-01 00:00:00.000000   
3  8a858efd5ca70688015cabd1f1e94b55  1991-07-19 00:00:00.000000   
4  8a858e785acd3412015acd48f4920d04  1982-11-22 00:00:00.000000   

  bank_account_type  longitude_gps  latitude_gps bank_name_clients  \
0           Savings       3.319219      6.528604           GT Bank   
1           Savings       3.325598      7.119403     Sterling Bank   
2           Savings       5.746100      5.563174     Fidelity Bank   
3           Savings       3.362850      6.642485           GT Bank   
4           Savings       8.455332     11.971410           GT Bank   

  bank_branch_clients employment_status_clients level_of_education_clients  
0                 NaN                       NaN                        NaN  
1                 NaN 

In [4]:
data_one = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/trainperf.csv')
print(data_one.head())

                         customerid  systemloanid  loannumber  \
0  8a2a81a74ce8c05d014cfb32a0da1049     301994762          12   
1  8a85886e54beabf90154c0a29ae757c0     301965204           2   
2  8a8588f35438fe12015444567666018e     301966580           7   
3  8a85890754145ace015429211b513e16     301999343           3   
4  8a858970548359cc0154883481981866     301962360           9   

                 approveddate                creationdate  loanamount  \
0  2017-07-25 08:22:56.000000  2017-07-25 07:22:47.000000     30000.0   
1  2017-07-05 17:04:41.000000  2017-07-05 16:04:18.000000     15000.0   
2  2017-07-06 14:52:57.000000  2017-07-06 13:52:51.000000     20000.0   
3  2017-07-27 19:00:41.000000  2017-07-27 18:00:35.000000     10000.0   
4  2017-07-03 23:42:45.000000  2017-07-03 22:42:39.000000     40000.0   

   totaldue  termdays referredby good_bad_flag  
0   34500.0        30        NaN          Good  
1   17250.0        30        NaN          Good  
2   22250.0        15  

In [5]:
data_two = pd.read_csv('/content/drive/MyDrive/Classroom/loan_data/trainprevloans.csv')
print(data_two.head())

                         customerid  systemloanid  loannumber  \
0  8a2a81a74ce8c05d014cfb32a0da1049     301682320           2   
1  8a2a81a74ce8c05d014cfb32a0da1049     301883808           9   
2  8a2a81a74ce8c05d014cfb32a0da1049     301831714           8   
3  8a8588f35438fe12015444567666018e     301861541           5   
4  8a85890754145ace015429211b513e16     301941754           2   

                 approveddate                creationdate  loanamount  \
0  2016-08-15 18:22:40.000000  2016-08-15 17:22:32.000000     10000.0   
1  2017-04-28 18:39:07.000000  2017-04-28 17:38:53.000000     10000.0   
2  2017-03-05 10:56:25.000000  2017-03-05 09:56:19.000000     20000.0   
3  2017-04-09 18:25:55.000000  2017-04-09 17:25:42.000000     10000.0   
4  2017-06-17 09:29:57.000000  2017-06-17 08:29:50.000000     10000.0   

   totaldue  termdays                  closeddate referredby  \
0   13000.0        30  2016-09-01 16:06:48.000000        NaN   
1   13000.0        30  2017-05-28 14:44:49

In [6]:
print(data.tail())

                            customerid                   birthdate  \
4341  8a858f155554552501555588ca2b3b40  1985-12-13 00:00:00.000000   
4342  8a858fc65cf978f4015cf97cee3a02ce  1982-07-01 00:00:00.000000   
4343  8a858f4f5b66de3a015b66fc83c61902  1989-09-26 00:00:00.000000   
4344  8aaae7a74400b28201441c8b62514150  1985-09-06 00:00:00.000000   
4345  8a85896653e2e18b0153e69c1b90265c  1975-06-05 00:00:00.000000   

     bank_account_type  longitude_gps  latitude_gps bank_name_clients  \
4341             Other       3.236753      7.030168      Stanbic IBTC   
4342           Savings       7.013750      4.875662           GT Bank   
4343           Savings       6.295530      7.092508           GT Bank   
4344           Savings       3.354206      6.539070           GT Bank   
4345           Savings       6.661014      7.472700               UBA   

     bank_branch_clients employment_status_clients level_of_education_clients  
4341                 NaN                 Permanent          

In [7]:
data.dtypes

customerid                     object
birthdate                      object
bank_account_type              object
longitude_gps                 float64
latitude_gps                  float64
bank_name_clients              object
bank_branch_clients            object
employment_status_clients      object
level_of_education_clients     object
dtype: object

In [8]:
# data.info

In [9]:
data.shape

(4346, 9)

In [10]:
data.isnull().sum()

customerid                       0
birthdate                        0
bank_account_type                0
longitude_gps                    0
latitude_gps                     0
bank_name_clients                0
bank_branch_clients           4295
employment_status_clients      648
level_of_education_clients    3759
dtype: int64

In [11]:
data_dup = data.duplicated().any()
data_dup

True

In [12]:
dup_count = data.duplicated().sum()
dup_count

12

In [13]:
data = data.drop_duplicates()

In [14]:
data.shape

(4334, 9)

In [15]:
dup_count = data.duplicated().sum()

In [16]:
data = data.drop(columns=['bank_branch_clients'])

In [17]:
data.shape

(4334, 8)

In [18]:
print(data.head())

                         customerid                   birthdate  \
0  8a858e135cb22031015cbafc76964ebd  1973-10-10 00:00:00.000000   
1  8a858e275c7ea5ec015c82482d7c3996  1986-01-21 00:00:00.000000   
2  8a858e5b5bd99460015bdc95cd485634  1987-04-01 00:00:00.000000   
3  8a858efd5ca70688015cabd1f1e94b55  1991-07-19 00:00:00.000000   
4  8a858e785acd3412015acd48f4920d04  1982-11-22 00:00:00.000000   

  bank_account_type  longitude_gps  latitude_gps bank_name_clients  \
0           Savings       3.319219      6.528604           GT Bank   
1           Savings       3.325598      7.119403     Sterling Bank   
2           Savings       5.746100      5.563174     Fidelity Bank   
3           Savings       3.362850      6.642485           GT Bank   
4           Savings       8.455332     11.971410           GT Bank   

  employment_status_clients level_of_education_clients  
0                       NaN                        NaN  
1                 Permanent                        NaN  
2    

In [19]:
arr1 = data['customerid'].to_numpy()
arr2 = data_two['customerid'].to_numpy()

In [20]:
intersection = np.intersect1d(arr1, arr2)

In [21]:
print(intersection)

['8a1088a0484472eb01484669e3ce4e0b' '8a1a1e7e4f707f8b014f797718316cad'
 '8a1a32fc49b632520149c3b8fdf85139' ... '8a858fff5a36fe68015a37f8550a02a2'
 '8a858fff5a36fe68015a3b8dcb8a3843' '8a858fff5c79144c015c7bdbfc086ce1']
